# Segmenting and Clustering Neighborhoods in Toronto
### CAPSTONE PROJECT WEEK 3

For this assignment, we will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.


Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [16]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

* ##### Use beautifulsoup to scrape the postal code information from the wikipedia link given above

In [17]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, "lxml")
table = soup.find("table")
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

* ##### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [18]:
df = pd.DataFrame(res, columns=["Postal Code", "Borough", "Neighborhood"])
df.dtypes

Postal Code     object
Borough         object
Neighborhood    object
dtype: object

* ##### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [19]:
df = df[(df.Borough != 'Not assigned')] #& (df.Neighborhood !='None')]
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


* ##### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be  the same as the borough.

The neighborhoods with same postalcode have already been merged in the wikipedia page. Hence we only replace '/' with ',' to get the desired dataframe


In [20]:
df['Neighborhood'] = df['Neighborhood'].str.replace('/', ',')
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [21]:
df.shape

(103, 3)

In [22]:
import geocoder
from geopy.geocoders import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium

In [25]:
df_latlng = pd.read_csv('Geospatial_Coordinates.csv')
df_latlng.shape

(103, 3)

In [13]:
df['Latitude'] = geocoder.google('{}, Toronto, Ontario'.format(df["PostalCode"]).latlng[0]
df['Longitude'] = geocoder.google('{}, Toronto, Ontario'.format(df["PostalCode"]).latlng[1]
df

SyntaxError: invalid syntax (<ipython-input-13-8fff80809495>, line 2)

In [29]:
df_loc = pd.merge(df, df_latlng, on='Postal Code')
df_loc.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
